<a href="https://colab.research.google.com/github/fumio125/ou_dip/blob/master/ou_dip_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np  # PythonのOpenCVでは、画像はnumpyのarrayとして管理される
from google.colab.patches import cv2_imshow # colab内で画像表示関数がうまく動かないので、パッチが提供されている

# Googleドライブへのマウント
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/ou_dip/"

In [2]:
def createSpatialGaussian(size):
  # size: 横幅（=縦幅）。5x5なら5。奇数にすること
  # 半径 ((size-1)/2) の半分、としてガウス分布の標準偏差sigmaを設定（5x5のとき、sigma=1）
  sigma = (size-1) / 4
  gauss_1d = cv2.getGaussianKernel(size, sigma) 
  gauss_2d = np.matmul(gauss_1d, gauss_1d.T)

  return gauss_2d

def createFrequencyGaussian(spatial_filter, img):
  # 空間フィルタをDFTして周波数フィルタを生成（他の方法もあるが、実装が楽なので）
  # spatial_filter: 空間フィルタ, img: 入力画像
  gauss_freq = np.zeros(img.shape)
  gauss_freq[0:size,0:size] = spatial_filter  # 左上にコピー
  gauss_freq = cv2.dft(gauss_freq.astype(np.float32), flags=cv2.DFT_COMPLEX_OUTPUT) # 1ch目に実部、2ch目に虚数部

  # 可視化用
  re, im = cv2.split(gauss_freq)
  gauss_freq = cv2.magnitude(re, im)
  gauss_freq = gauss_freq / np.max(gauss_freq)

  return gauss_freq

# numpyのみを使ったconvolve2dの実装（einsum）
# https://stackoverflow.com/questions/43086557/convolve2d-just-by-using-numpy
# https://qiita.com/secang0/items/f3a3ff629988dc660d87
def convolve2d(img, kernel):
    #部分行列の大きさを計算
    sub_shape = tuple(np.subtract(img.shape, kernel.shape) + 1)

    #部分行列の行列を作成
    submatrices = np.lib.stride_tricks.as_strided(img,kernel.shape + sub_shape,img.strides * 2)

    #部分行列とカーネルのアインシュタイン和を計算
    convolved_matrix = np.einsum('ij,ijkl->kl', kernel, submatrices)

    return convolved_matrix

In [ ]:
# 準備：画像の読み込みとフィルタの生成
size = 11 # フィルタサイズ

#src = cv2.imread("sample.jpg", cv2.IMREAD_GRAYSCALE) # 画像読み込み
src = cv2.imread("pano_ref.jpg", cv2.IMREAD_GRAYSCALE) # 画像読み込み

spatial_filter = createSpatialGaussian(size)
print('Spatial filter:')
cv2_imshow(spatial_filter / np.max(spatial_filter) * 255) # 最大値が255になるようにして表示

print('Frequency filter:')
freq_filter = createFrequencyGaussian(size, src)  # 左上原点！
cv2_imshow(freq_filter  * 255) # 最大値が255になるようにして表示


In [ ]:
# 空間フィルタ
from scipy import signal
import time

start_time = time.perf_counter()

dst = convolve2d(src,spatial_filter)  # 外部領域は切られる実装

elapsed_time = time.perf_counter() - start_time
print('spatial filtering:', elapsed_time, '[sec]')

cv2_imshow(dst)

In [ ]:
# 周波数フィルタ
import time
start_time = time.perf_counter()

src_freq = cv2.dft(src.astype(np.float32), flags=cv2.DFT_COMPLEX_OUTPUT) # 1ch目に実部、2ch目に虚数部 にしたい場合


dst_freq = src_freq * np.dstack((freq_filter,freq_filter))  # 実部・虚数部両方に掛ける
dst = cv2.idft(dst_freq, flags=cv2.DFT_SCALE)[:,:,0]  # 逆フーリエ変換

elapsed_time = time.perf_counter() - start_time
print('frequency filtering:', elapsed_time, '[sec]')

# magnitudeの可視化
re, im = cv2.split(src_freq)
magnitude = cv2.magnitude(re, im)
cv2_imshow(magnitude / np.mean(magnitude)*10)

cv2_imshow(dst)

In [ ]:
# SciPy (Optimized)
import time

start_time = time.perf_counter()

dst = signal.correlate(src,spatial_filter,'same') # 画像の外側は0になる

elapsed_time = time.perf_counter() - start_time
print('Scipy filtering (optimized):', elapsed_time, '[sec]')

cv2_imshow(dst)

In [ ]:
# OpenCV (Optimized)
import time

start_time = time.perf_counter()

dst = cv2.filter2D(src,-1,spatial_filter) # ddepth=-1とした場合、入出力のchannel数はおなじになる

elapsed_time = time.perf_counter() - start_time
print('OpenCV filtering (optimized):', elapsed_time, '[sec]')

cv2_imshow(dst)

In [8]:
# 比較用に関数化
import time

def spatialFiltering(src, spatial_filter):
  start_time = time.perf_counter()
  dst = convolve2d(src,spatial_filter)  # 外部領域は切られる実装
  elapsed_time = time.perf_counter() - start_time
  return elapsed_time

def frequencyFiltering(src, freq_filter):
  start_time = time.perf_counter()
  src_freq = cv2.dft(src.astype(np.float32), flags=cv2.DFT_COMPLEX_OUTPUT) # 1ch目に実部、2ch目に虚数部 にしたい場合
  dst_freq = src_freq * np.dstack((freq_filter,freq_filter))  # 実部・虚数部両方に掛ける
  dst = cv2.idft(dst_freq, flags=cv2.DFT_SCALE)[:,:,0]  # 逆フーリエ変換
  elapsed_time = time.perf_counter() - start_time
  return elapsed_time

def SciPyFiltering(src, spatial_filter):
  start_time = time.perf_counter()
  dst = signal.correlate(src,spatial_filter,'same') # 画像の外側は0になる
  elapsed_time = time.perf_counter() - start_time
  return elapsed_time

def OpenCVFiltering(sec, spatial_filter):
  start_time = time.perf_counter()
  dst = cv2.filter2D(src,-1,spatial_filter) # ddepth=-1とした場合、入出力のchannel数はおなじになる
  elapsed_time = time.perf_counter() - start_time
  return elapsed_time

In [ ]:
# フィルタサイズを変えながら比較

x = []
spatial_time = []
freq_time = []
scipy_time = []
opencv_time = []

#src = cv2.imread("sample.jpg", cv2.IMREAD_GRAYSCALE) # 画像読み込み
src = cv2.imread("pano_ref.jpg", cv2.IMREAD_GRAYSCALE) # 画像読み込み

for i in range(3,31,2):  
  print('filter size:',i)
  spatial_filter = createSpatialGaussian(i)
  freq_filter = createFrequencyGaussian(i, src)  

  spatial_time.append(spatialFiltering(src, spatial_filter))
  freq_time.append(frequencyFiltering(src,freq_filter))
  scipy_time.append(SciPyFiltering(src, spatial_filter))
  opencv_time.append(OpenCVFiltering(src, spatial_filter))
  x.append(i)

In [ ]:
# 描画

%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
x_axis = np.array(x)
ax.plot(x_axis, np.array(spatial_time), label='spatial')
ax.plot(x_axis, np.array(freq_time), label='freq')
ax.plot(x_axis, np.array(scipy_time), label='scipy')
ax.plot(x_axis, np.array(opencv_time), label='opencv')
ax.legend(loc=0)    # 凡例